# Overview
This notebook has the goal of opening and preprocessing the climate data associated with the midwest region of the United States. The data is composed of 9 datasets: 

* tp: Total precipitation
* sst: Sea surface temperature
* thickness: Tropospheric thickness between 850 and 500 hPa
* pottemp: 45m ocean potential temperature
* pr_wtr: Precipitable water
* lftx4: Atmospheric lifted index
* chi: upper troposphere velocity potential
* tcdc: total cloud cover
* shum: 2m specific humidity

The data is available in the form of NetCDF files, which are opened using the `xarray` library. We have the initial challenge that the predictior and target datasets have different resolutions and alignments. This data needs to be resampled and subset in order for the analysis to take place. Because Sea Surface Temperature (SST) and Ocean Potential Temperature (pottemp) do not have any overlap with the area of interest, they are replaced with the Ocean Nino Index (ONI) which can give us context about global phenomena that affect precipitation.

The preprocessing steps are as follows:
1. Open the data
2. Subset predictor data to the region of interest
3. Resample precipitation data to the resolution of the predictor data
4. Merge the datasets
5. Save the data in a format that can be used in the next steps

In [1]:
import xarray as xr
import pandas as pd
import pathlib
import os

In [2]:
def get_netcdf_metadata(dataset: xr.Dataset):
    """
    Get the bounding box coordinates, origin, and resolution from a NetCDF dataset.

    Parameters:
        dataset (xr.Dataset): The xarray dataset.

    Returns:
        dict: A dictionary containing bounding box coordinates
              (lat_bottom, lat_top, lon_left, lon_right).
        tuple: The origin (first latitude, first longitude).
        float: Latitude resolution.
        float: Longitude resolution.
    """
    # Get the latitude and longitude arrays (assuming geospatial data)
    lat = dataset["latitude"].values
    lon = dataset["longitude"].values  # same for 'longitude'

    # Get bounding box coordinates
    bounding_box = {
        "lat_bottom": lat.min(),  # southern limit
        "lat_top": lat.max(),  # northern limit
        "lon_left": lon.min(),  # western limit
        "lon_right": lon.max(),  # eastern limit
    }

    # Determine the origin (first lat/lon value)
    origin = (lat[0], lon[0])

    # Calculate resolution (difference between consecutive latitude and longitude values)
    lat_resolution = abs(lat[1] - lat[0])
    lon_resolution = abs(lon[1] - lon[0])

    return bounding_box, origin, lat_resolution, lon_resolution

# Load Data

In [3]:
# get the data directory
data_dir = pathlib.Path(os.getcwd()).parent / "data"
precip_file = data_dir / "raw_data" / "monthly_us_midwest_precip.nc"
predictor_file = data_dir / "raw_data" / "predictor_variables.nc"
oni_file = data_dir / "raw_data" / "ocean_nino_index.csv"

In [4]:
# open the datasets
ds_precipitation = xr.open_dataset(precip_file)
ds_predictors = xr.open_dataset(predictor_file)

# Bounding Boxes

In [5]:
# get the metadata
precip_bounding_box, precip_origin, precip_res_y, precip_res_x = get_netcdf_metadata(
    ds_precipitation
)

In [6]:
# get the metadata
pred_bounding_box, pred_origin, pred_res_y, pred_res_x = get_netcdf_metadata(
    ds_predictors
)

# Resample Data

In [7]:
# Subset predictor dataset with buffer
ds_predictors_subset = ds_predictors.sel(
    latitude=slice(
        precip_bounding_box["lat_bottom"],
        precip_bounding_box["lat_top"],
    ),
    longitude=slice(
        precip_bounding_box["lon_left"],
        precip_bounding_box["lon_right"],
    ),
)

In [8]:
# Resample the predictor dataset to 0.25° resolution using interpolation
ds_precipitation_resampled = ds_precipitation.interp(
    latitude=ds_predictors_subset["latitude"],
    longitude=ds_predictors_subset["longitude"],
    method="linear",  # interpolation
)

# Merge Data

In [9]:
# Combine the resampled predictors with the precipitation dataset
combined_ds = xr.merge(
    [
        ds_precipitation_resampled,
        ds_predictors_subset,
    ]
)

In [10]:
print(combined_ds)

<xarray.Dataset> Size: 1MB
Dimensions:    (time: 504, latitude: 5, longitude: 13)
Coordinates:
  * time       (time) datetime64[ns] 4kB 1982-01-01 1982-02-01 ... 2023-12-01
  * latitude   (latitude) float32 20B 37.5 40.0 42.5 45.0 47.5
  * longitude  (longitude) float32 52B -110.0 -107.5 -105.0 ... -82.5 -80.0
    level      float32 4B 45.0
Data variables:
    tp         (time, latitude, longitude) float32 131kB 45.93 130.2 ... 26.05
    sst        (time, latitude, longitude) float32 131kB ...
    thickness  (time, latitude, longitude) float32 131kB ...
    pottmp     (time, latitude, longitude) float32 131kB ...
    pr_wtr     (time, latitude, longitude) float32 131kB ...
    lftx4      (time, latitude, longitude) float32 131kB ...
    chi        (time, latitude, longitude) float32 131kB ...
    tcdc       (time, latitude, longitude) float32 131kB ...
    shum       (time, latitude, longitude) float32 131kB ...


In [11]:
# remove Remove null dataset because it doesn't overlap with region of interest
combined_ds = combined_ds.drop_vars(["sst", "pottmp"])

# Save the combined dataset to a new netCDF file
combined_ds.to_netcdf(data_dir / "processed_data" / "combined_data.nc")

# Cold & Warm Episodes by Season
This section contains the Oceanic Niño Index (ONI) values for each month, which are used to identify warm and cold episodes associated with the El Niño-Southern Oscillation (ENSO).

Notice: Due to the high-frequency filter applied to the ERSSTv5 data (Huang et al., 2017, Journal of Climate), ONI values may change up to two months after the initial "real-time" value is posted. Therefore, the most recent ONI values should be regarded as estimates.

Description
The ONI indicates warm (red) and cold (blue) periods based on a threshold of ±0.5°C. It represents a 3-month running mean of ERSST.v5 SST anomalies in the Niño 3.4 region (5°N–5°S, 120°–170°W), calculated using centered 30-year base periods that are updated every 5 years.

For historical purposes, periods of below-normal and above-normal sea surface temperatures (SSTs) are color-coded in blue and red, respectively, when the threshold is met for a minimum of 5 consecutive overlapping seasons.

The ONI is one measure of the El Niño-Southern Oscillation, and other indices can confirm whether features consistent with a coupled ocean-atmosphere phenomenon accompany these periods.

For more information, visit the NOAA CPC ONI page. https://origin.cpc.ncep.noaa.gov/products/analysis_monitoring/ensostuff/ONI_v5.php

In [12]:
# load oni data
def oni_pipeline(filepath: pathlib.Path):
    # read data
    df = pd.read_csv(filepath, header=0, index_col=0, sep=",")

    # remove rows with "Year" in index
    df = df[df.index != "Year"]

    # flatten dataframe to 1 column
    df = df.stack().reset_index()

    # make pandas datetime index from january 1950 to december 2022
    df.index = pd.date_range(start="1950-01-01", end="2024-07-01", freq="MS")

    # drop columns
    df.drop(["Year", "level_1"], axis=1, inplace=True)

    # rename columns
    df.rename(columns={0: "oni"}, inplace=True)

    # make enso column numeric
    df["oni"] = pd.to_numeric(df["oni"]).astype(float)

    return df

In [13]:
# Clean ONI dataset and save to CSV
oni = oni_pipeline(oni_file)
oni.to_csv(data_dir / "processed_data" / "ocean_nino_index.csv")